# Firefox Session Cleanup on Server Shutdown

This notebook demonstrates how to ensure all Xpra/Firefox sessions created by the notebook are properly cleaned up when the notebook server is culled or terminated.

## Key Challenges:
1. **Server Culling**: When a notebook server is culled by JupyterHub or the spawner, processes may be left running
2. **Orphaned Sessions**: Xpra and Firefox processes can become orphaned if not properly terminated
3. **Resource Cleanup**: Session directories, sockets, and temp files need to be cleaned up
4. **Signal Handling**: Different termination signals (SIGTERM, SIGINT, SIGKILL) require different strategies

## Our Solution:
- **Signal Handlers**: Register cleanup functions for SIGTERM and SIGINT
- **atexit Hooks**: Ensure cleanup runs even on normal Python exit
- **Jupyter Server Hooks**: Integrate with Jupyter server shutdown process
- **Process Tracking**: Maintain a registry of all active sessions for comprehensive cleanup

## 1. Import Required Libraries

Import necessary libraries for signal handling, process management, and cleanup functionality.

In [1]:
import os
import sys
import signal
import atexit
import logging
import traceback
import threading
import time
from pathlib import Path
from typing import Dict, List, Set, Any

import psutil

# Configure logging for cleanup operations
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

cleanup_logger = logging.getLogger('firefox_cleanup')
cleanup_logger.info("🧹 Firefox session cleanup system initialized")

2025-07-28 14:56:21,700 - firefox_cleanup - INFO - 🧹 Firefox session cleanup system initialized


## 2. Track Active Xpra/Firefox Sessions

Create a centralized registry to track all active Firefox/Xpra sessions that need cleanup when the server shuts down.

In [2]:
class FirefoxSessionRegistry:
    """Centralized registry for tracking active Firefox/Xpra sessions."""
    
    def __init__(self):
        self._sessions: Dict[int, Dict[str, Any]] = {}  # port -> session_info
        self._lock = threading.Lock()
        self._cleanup_registered = False
        
    def register_session(self, port: int, process_id: int, session_dir: Path = None):
        """Register a new Firefox/Xpra session for cleanup tracking."""
        with self._lock:
            self._sessions[port] = {
                'process_id': process_id,
                'port': port,
                'session_dir': session_dir,
                'registered_at': time.time()
            }
            cleanup_logger.info(f"📝 Registered session: port={port}, pid={process_id}")
            
            # Register cleanup handlers on first session
            if not self._cleanup_registered:
                self._register_cleanup_handlers()
                self._cleanup_registered = True
                
    def unregister_session(self, port: int):
        """Remove a session from cleanup tracking."""
        with self._lock:
            if port in self._sessions:
                session_info = self._sessions.pop(port)
                cleanup_logger.info(f"📝 Unregistered session: port={port}, pid={session_info.get('process_id')}")
                
    def get_active_sessions(self) -> Dict[int, Dict[str, Any]]:
        """Get a copy of all currently tracked sessions."""
        with self._lock:
            return dict(self._sessions)
            
    def cleanup_all_sessions(self, force: bool = False):
        """Clean up all registered sessions."""
        with self._lock:
            sessions_to_clean = list(self._sessions.items())
            
        if not sessions_to_clean:
            cleanup_logger.info("🧹 No active sessions to clean up")
            return 0
            
        cleanup_logger.info(f"🧹 Cleaning up {len(sessions_to_clean)} active sessions (force={force})")
        cleaned = 0
        
        for port, session_info in sessions_to_clean:
            try:
                process_id = session_info['process_id']
                session_dir = session_info.get('session_dir')
                
                # Terminate process and children
                if self._terminate_process_tree(process_id, force):
                    cleaned += 1
                    
                # Clean up session directory
                if session_dir and session_dir.exists():
                    try:
                        import shutil
                        shutil.rmtree(session_dir)
                        cleanup_logger.info(f"🗑️ Removed session directory: {session_dir}")
                    except Exception as dir_error:
                        cleanup_logger.warning(f"⚠️ Failed to remove session directory {session_dir}: {dir_error}")
                        
                # Remove from tracking
                self.unregister_session(port)
                
            except Exception as session_error:
                cleanup_logger.error(f"❌ Error cleaning up session on port {port}: {session_error}")
                
        cleanup_logger.info(f"✅ Cleaned up {cleaned}/{len(sessions_to_clean)} sessions")
        return cleaned
        
    def _terminate_process_tree(self, process_id: int, force: bool = False) -> bool:
        """Terminate a process and all its children."""
        try:
            process = psutil.Process(process_id)
            process_name = process.name()
            
            # Get all child processes first
            children = process.children(recursive=True)
            cleanup_logger.info(f"🔥 Terminating process tree: PID {process_id} ({process_name}) + {len(children)} children")
            
            # Terminate children first
            for child in children:
                try:
                    child.terminate()
                    cleanup_logger.debug(f"   Terminated child: {child.pid}")
                except (psutil.NoSuchProcess, psutil.AccessDenied):
                    pass
                    
            # Terminate main process
            process.terminate()
            
            # Wait for graceful termination
            timeout = 3 if not force else 1
            try:
                process.wait(timeout=timeout)
                cleanup_logger.info(f"✅ Process {process_id} terminated gracefully")
                return True
            except psutil.TimeoutExpired:
                if force:
                    # Force kill if needed
                    cleanup_logger.warning(f"⏰ Force killing unresponsive process {process_id}")
                    process.kill()
                    for child in children:
                        try:
                            child.kill()
                        except (psutil.NoSuchProcess, psutil.AccessDenied):
                            pass
                    return True
                else:
                    cleanup_logger.warning(f"⏰ Process {process_id} didn't terminate in {timeout}s")
                    return False
                    
        except psutil.NoSuchProcess:
            cleanup_logger.info(f"⚠️ Process {process_id} already terminated")
            return True
        except Exception as term_error:
            cleanup_logger.error(f"❌ Error terminating process {process_id}: {term_error}")
            return False
            
    def _register_cleanup_handlers(self):
        """Register cleanup handlers for various shutdown scenarios."""
        cleanup_logger.info("🔧 Registering cleanup handlers for server shutdown")
        
        # Register atexit handler
        atexit.register(self._atexit_cleanup)
        
        # Register signal handlers
        signal.signal(signal.SIGTERM, self._signal_cleanup)
        signal.signal(signal.SIGINT, self._signal_cleanup)
        
        cleanup_logger.info("✅ Cleanup handlers registered successfully")
        
    def _atexit_cleanup(self):
        """Cleanup handler for normal Python exit."""
        cleanup_logger.info("🚪 atexit cleanup triggered - cleaning up all sessions")
        self.cleanup_all_sessions(force=True)
        
    def _signal_cleanup(self, signum, frame):
        """Cleanup handler for signal-based termination."""
        signal_name = signal.Signals(signum).name
        cleanup_logger.info(f"📡 Signal {signal_name} received - cleaning up all sessions")
        self.cleanup_all_sessions(force=True)
        
        # Re-raise the signal for normal termination
        signal.signal(signum, signal.SIG_DFL)
        os.kill(os.getpid(), signum)

# Create global session registry
session_registry = FirefoxSessionRegistry()
cleanup_logger.info("🏭 Global Firefox session registry created")

2025-07-28 14:56:27,166 - firefox_cleanup - INFO - 🏭 Global Firefox session registry created


## 3. Jupyter Server Integration

Integrate with Jupyter server shutdown hooks to ensure cleanup happens when the server is terminated by JupyterHub or the spawner.

In [3]:
def register_jupyter_server_cleanup():
    """Register cleanup with Jupyter server if available."""
    try:
        # Try to get the current Jupyter server instance
        from jupyter_server.serverapp import ServerApp
        from jupyter_core.application import JupyterApp
        
        # Get the current server app instance
        app_instance = None
        if hasattr(JupyterApp, 'instance'):
            app_instance = JupyterApp.instance()
            
        if app_instance and hasattr(app_instance, 'add_hook'):
            # Register shutdown hook with Jupyter server
            app_instance.add_hook('shutdown', session_registry.cleanup_all_sessions)
            cleanup_logger.info("✅ Registered cleanup with Jupyter server shutdown hooks")
            return True
        else:
            cleanup_logger.warning("⚠️ Could not register with Jupyter server hooks (no app instance)")
            return False
            
    except ImportError:
        cleanup_logger.warning("⚠️ Jupyter server not available for hook registration")
        return False
    except Exception as hook_error:
        cleanup_logger.warning(f"⚠️ Failed to register Jupyter server hooks: {hook_error}")
        return False

def create_pid_file():
    """Create a PID file to help external cleanup scripts."""
    try:
        pid_file = Path.home() / '.firefox-launcher' / 'jupyter_server.pid'
        pid_file.parent.mkdir(parents=True, exist_ok=True)
        
        with open(pid_file, 'w') as f:
            f.write(str(os.getpid()))
            
        cleanup_logger.info(f"📄 Created PID file: {pid_file}")
        
        # Register cleanup for PID file
        def cleanup_pid_file():
            try:
                if pid_file.exists():
                    pid_file.unlink()
                    cleanup_logger.info(f"🗑️ Removed PID file: {pid_file}")
            except Exception as pid_error:
                cleanup_logger.warning(f"⚠️ Failed to remove PID file: {pid_error}")
                
        atexit.register(cleanup_pid_file)
        return pid_file
        
    except Exception as pid_error:
        cleanup_logger.warning(f"⚠️ Failed to create PID file: {pid_error}")
        return None

# Register with Jupyter server
jupyter_hooks_registered = register_jupyter_server_cleanup()

# Create PID file for external cleanup
pid_file = create_pid_file()

cleanup_logger.info(f"🔧 Jupyter integration: hooks_registered={jupyter_hooks_registered}, pid_file={pid_file}")

2025-07-28 14:56:34,064 - firefox_cleanup - WARNING - ⚠️ Failed to register Jupyter server hooks: An incompatible sibling of 'JupyterApp' is already instantiated as singleton: IPKernelApp
2025-07-28 14:56:34,065 - firefox_cleanup - INFO - 📄 Created PID file: /home/bdx/.firefox-launcher/jupyter_server.pid
2025-07-28 14:56:34,065 - firefox_cleanup - INFO - 🔧 Jupyter integration: hooks_registered=False, pid_file=/home/bdx/.firefox-launcher/jupyter_server.pid


## 4. Test Session Cleanup System

Test the cleanup system by simulating sessions and verifying cleanup works correctly.

In [4]:
# Test session registration
def test_session_registration():
    """Test session registration and tracking."""
    cleanup_logger.info("🧪 Testing session registration...")
    
    # Simulate registering sessions (using dummy PIDs for testing)
    test_sessions = [
        (8080, 12345),
        (8081, 12346),
        (8082, 12347)
    ]
    
    for port, pid in test_sessions:
        session_dir = Path.home() / '.firefox-launcher' / 'sessions' / f'session-{port}'
        session_registry.register_session(port, pid, session_dir)
        
    # Check active sessions
    active = session_registry.get_active_sessions()
    cleanup_logger.info(f"📊 Active sessions after registration: {len(active)}")
    
    for port, info in active.items():
        cleanup_logger.info(f"   Port {port}: PID {info['process_id']}")
        
    return active

# Test manual cleanup
def test_manual_cleanup():
    """Test manual cleanup of registered sessions."""
    cleanup_logger.info("🧪 Testing manual cleanup...")
    
    active_before = len(session_registry.get_active_sessions())
    cleanup_logger.info(f"📊 Sessions before cleanup: {active_before}")
    
    # Perform cleanup (will safely handle non-existent PIDs)
    cleaned = session_registry.cleanup_all_sessions(force=True)
    
    active_after = len(session_registry.get_active_sessions())
    cleanup_logger.info(f"📊 Sessions after cleanup: {active_after}")
    cleanup_logger.info(f"✅ Test completed: {cleaned} sessions processed")

# Run tests
test_sessions = test_session_registration()
print(f"Registered {len(test_sessions)} test sessions")

2025-07-28 14:56:39,735 - firefox_cleanup - INFO - 🧪 Testing session registration...
2025-07-28 14:56:39,736 - firefox_cleanup - INFO - 📝 Registered session: port=8080, pid=12345
2025-07-28 14:56:39,737 - firefox_cleanup - INFO - 🔧 Registering cleanup handlers for server shutdown
2025-07-28 14:56:39,738 - firefox_cleanup - INFO - ✅ Cleanup handlers registered successfully
2025-07-28 14:56:39,739 - firefox_cleanup - INFO - 📝 Registered session: port=8081, pid=12346
2025-07-28 14:56:39,739 - firefox_cleanup - INFO - 📝 Registered session: port=8082, pid=12347
2025-07-28 14:56:39,740 - firefox_cleanup - INFO - 📊 Active sessions after registration: 3
2025-07-28 14:56:39,741 - firefox_cleanup - INFO -    Port 8080: PID 12345
2025-07-28 14:56:39,741 - firefox_cleanup - INFO -    Port 8081: PID 12346
2025-07-28 14:56:39,742 - firefox_cleanup - INFO -    Port 8082: PID 12347


Registered 3 test sessions


In [5]:
# Test the cleanup system
test_manual_cleanup()

# Display current system status
print("\n" + "="*60)
print("🔧 FIREFOX SESSION CLEANUP SYSTEM STATUS")
print("="*60)
print(f"✅ Cleanup handlers registered: {session_registry._cleanup_registered}")
print(f"✅ Jupyter hooks registered: {jupyter_hooks_registered}")
print(f"✅ PID file created: {pid_file is not None}")
print(f"📊 Active sessions: {len(session_registry.get_active_sessions())}")
print("\n🛡️ PROTECTION MECHANISMS:")
print("   • atexit handlers for normal Python exit")
print("   • Signal handlers for SIGTERM/SIGINT")
print("   • Jupyter server shutdown hooks (if available)")
print("   • PID file for external cleanup scripts")
print("\n🧹 CLEANUP FEATURES:")
print("   • Process tree termination (parent + children)")
print("   • Session directory cleanup")
print("   • Graceful shutdown with force fallback")
print("   • Thread-safe session tracking")

2025-07-28 14:56:45,466 - firefox_cleanup - INFO - 🧪 Testing manual cleanup...
2025-07-28 14:56:45,467 - firefox_cleanup - INFO - 📊 Sessions before cleanup: 3
2025-07-28 14:56:45,468 - firefox_cleanup - INFO - 🧹 Cleaning up 3 active sessions (force=True)
2025-07-28 14:56:45,469 - firefox_cleanup - INFO - ⚠️ Process 12345 already terminated
2025-07-28 14:56:45,470 - firefox_cleanup - INFO - 📝 Unregistered session: port=8080, pid=12345
2025-07-28 14:56:45,471 - firefox_cleanup - INFO - ⚠️ Process 12346 already terminated
2025-07-28 14:56:45,472 - firefox_cleanup - INFO - 📝 Unregistered session: port=8081, pid=12346
2025-07-28 14:56:45,472 - firefox_cleanup - INFO - ⚠️ Process 12347 already terminated
2025-07-28 14:56:45,473 - firefox_cleanup - INFO - 📝 Unregistered session: port=8082, pid=12347
2025-07-28 14:56:45,473 - firefox_cleanup - INFO - ✅ Cleaned up 3/3 sessions
2025-07-28 14:56:45,474 - firefox_cleanup - INFO - 📊 Sessions after cleanup: 0
2025-07-28 14:56:45,474 - firefox_cleanu


🔧 FIREFOX SESSION CLEANUP SYSTEM STATUS
✅ Cleanup handlers registered: True
✅ Jupyter hooks registered: False
✅ PID file created: True
📊 Active sessions: 0

🛡️ PROTECTION MECHANISMS:
   • atexit handlers for normal Python exit
   • Signal handlers for SIGTERM/SIGINT
   • Jupyter server shutdown hooks (if available)
   • PID file for external cleanup scripts

🧹 CLEANUP FEATURES:
   • Process tree termination (parent + children)
   • Session directory cleanup
   • Graceful shutdown with force fallback
   • Thread-safe session tracking


## 5. Integration with Firefox Handler

To integrate this cleanup system with the actual Firefox launcher, we need to modify the `firefox_handler.py` to use the session registry.

In [ ]:
# Example integration code for firefox_handler.py
integration_code = '''
# Add to the top of firefox_handler.py
from .session_cleanup import session_registry

class FirefoxLauncherHandler(JupyterHandler):
    def _start_server_proxy(self):
        # ... existing code ...
        
        if final_poll is None:
            # Register the session for cleanup on shutdown
            session_dir = Path.home() / '.firefox-launcher' / 'sessions' / f'session-{port}'
            session_registry.register_session(port, process.pid, session_dir)
            
            # Store in local tracking (existing code)
            FirefoxLauncherHandler._active_sessions[port] = {
                'process_id': process.pid,
                'port': port
            }
            
            return True, port, process.pid
            
    def _stop_firefox(self):
        # ... existing code for stopping sessions ...
        
        # Also unregister from cleanup registry
        for port in sessions_to_stop:
            session_registry.unregister_session(port)
'''

print("INTEGRATION EXAMPLE:")
print("="*50)
print(integration_code)

print("\n📋 KEY INTEGRATION POINTS:")
print("1. Import session_registry in firefox_handler.py")
print("2. Register sessions when Xpra processes are created")
print("3. Unregister sessions when they are manually stopped")
print("4. The cleanup system handles server shutdown automatically")

print("\n🔄 CLEANUP TRIGGER SCENARIOS:")
print("• Normal Python exit (atexit)")
print("• SIGTERM from process manager/spawner")
print("• SIGINT from user (Ctrl+C)")
print("• Jupyter server shutdown hooks")
print("• External cleanup via PID file")

## 6. Debug Multi-Session Issues

Let's investigate why multiple Xpra sessions aren't working simultaneously.

In [6]:
import subprocess
import json
import requests
from pathlib import Path

def debug_active_firefox_sessions():
    """Debug current Firefox/Xpra sessions and identify issues."""
    print("🔍 DEBUGGING MULTI-SESSION ISSUES")
    print("=" * 50)
    
    # 1. Check if firefox-handler is accessible
    print("\n1. 🔧 CHECKING FIREFOX HANDLER STATUS")
    try:
        # Try to get the session status
        response = requests.get('http://localhost:8888/firefox-launcher?status=check', timeout=5)
        print(f"   ✅ Firefox handler responded: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            print(f"   📊 Active sessions: {data.get('active_sessions', 0)}")
            print(f"   📊 Status: {data.get('status', 'unknown')}")
        else:
            print(f"   ⚠️ Handler returned status {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Cannot connect to Firefox handler: {e}")
        print("   💡 Is JupyterLab running? Try: jupyter lab --port=8888")
    
    # 2. Check for running Xpra processes
    print("\n2. 🔍 CHECKING RUNNING XPRA PROCESSES")
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
        xpra_processes = [line for line in result.stdout.split('\n') if 'xpra' in line.lower()]
        
        if xpra_processes:
            print(f"   ✅ Found {len(xpra_processes)} Xpra processes:")
            for i, proc in enumerate(xpra_processes[:5]):  # Show first 5
                print(f"   {i+1}. {proc.strip()}")
            if len(xpra_processes) > 5:
                print(f"   ... and {len(xpra_processes) - 5} more")
        else:
            print("   ⚠️ No Xpra processes currently running")
    except Exception as e:
        print(f"   ❌ Error checking processes: {e}")
    
    # 3. Check for Firefox processes
    print("\n3. 🦊 CHECKING FIREFOX PROCESSES")
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
        firefox_processes = [line for line in result.stdout.split('\n') if 'firefox' in line.lower()]
        
        if firefox_processes:
            print(f"   ✅ Found {len(firefox_processes)} Firefox processes:")
            for i, proc in enumerate(firefox_processes[:3]):  # Show first 3
                print(f"   {i+1}. {proc.strip()}")
            if len(firefox_processes) > 3:
                print(f"   ... and {len(firefox_processes) - 3} more")
        else:
            print("   ⚠️ No Firefox processes currently running")
    except Exception as e:
        print(f"   ❌ Error checking Firefox processes: {e}")
    
    # 4. Check port usage
    print("\n4. 🌐 CHECKING PORT USAGE")
    try:
        result = subprocess.run(['netstat', '-tuln'], capture_output=True, text=True)
        listening_ports = []
        
        for line in result.stdout.split('\n'):
            if 'LISTEN' in line and ('tcp' in line.lower() or 'tcp6' in line.lower()):
                parts = line.split()
                if len(parts) >= 4:
                    addr = parts[3]
                    if ':' in addr:
                        port = addr.split(':')[-1]
                        if port.isdigit():
                            port_num = int(port)
                            # Check for common Xpra port range (typically 8000-9000)
                            if 8000 <= port_num <= 9000:
                                listening_ports.append(port_num)
        
        if listening_ports:
            print(f"   ✅ Found {len(listening_ports)} potential Xpra ports: {sorted(listening_ports)}")
        else:
            print("   ⚠️ No Xpra ports (8000-9000) currently listening")
            
    except Exception as e:
        print(f"   ❌ Error checking ports: {e}")
    
    # 5. Check session directories
    print("\n5. 📁 CHECKING SESSION DIRECTORIES")
    try:
        session_base = Path.home() / '.firefox-launcher' / 'sessions'
        if session_base.exists():
            session_dirs = list(session_base.iterdir())
            if session_dirs:
                print(f"   ✅ Found {len(session_dirs)} session directories:")
                for session_dir in session_dirs[:5]:  # Show first 5
                    print(f"   📂 {session_dir.name}")
                if len(session_dirs) > 5:
                    print(f"   ... and {len(session_dirs) - 5} more")
            else:
                print("   ⚠️ No session directories found")
        else:
            print("   ⚠️ Session base directory doesn't exist")
    except Exception as e:
        print(f"   ❌ Error checking session directories: {e}")
    
    # 6. Check for common issues
    print("\n6. 🚨 COMMON MULTI-SESSION ISSUES")
    issues_found = []
    
    # Check if Xpra version supports multiple sessions
    try:
        result = subprocess.run(['xpra', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            version_output = result.stdout.strip()
            print(f"   📋 Xpra version: {version_output}")
            
            # Check for known problematic versions
            if 'v3.0' in version_output:
                issues_found.append("Xpra v3.0 has known multi-session issues")
        else:
            issues_found.append("Xpra not found or not working")
    except Exception:
        issues_found.append("Cannot determine Xpra version")
    
    # Check for display conflicts
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
        display_lines = [line for line in result.stdout.split('\n') if 'DISPLAY=' in line and 'xpra' in line.lower()]
        
        displays = set()
        for line in display_lines:
            if 'DISPLAY=' in line:
                # Extract display number
                display_part = line.split('DISPLAY=')[1].split()[0]
                displays.add(display_part)
        
        if len(displays) > 1:
            print(f"   ✅ Multiple displays detected: {sorted(displays)}")
        elif len(displays) == 1:
            print(f"   ⚠️ Only one display in use: {list(displays)[0]}")
            issues_found.append("Potential display number conflict")
        else:
            print("   ⚠️ No display information found")
            
    except Exception as e:
        issues_found.append(f"Cannot check display usage: {e}")
    
    if issues_found:
        print("\n🚨 POTENTIAL ISSUES IDENTIFIED:")
        for i, issue in enumerate(issues_found, 1):
            print(f"   {i}. {issue}")
    else:
        print("\n✅ No obvious issues detected")
        
    print("\n💡 DEBUGGING SUGGESTIONS:")
    print("   1. Try starting a single session first to verify basic functionality")
    print("   2. Check the firefox_handler.py logs for specific error messages")  
    print("   3. Verify that the startup lock isn't blocking session creation")
    print("   4. Check if port allocation is working correctly")
    print("   5. Test Xpra manually: 'xpra start :100 --bind-tcp=0.0.0.0:8080'")

# Run the debugging function
debug_active_firefox_sessions()

🔍 DEBUGGING MULTI-SESSION ISSUES

1. 🔧 CHECKING FIREFOX HANDLER STATUS
   ✅ Firefox handler responded: 404
   ⚠️ Handler returned status 404

2. 🔍 CHECKING RUNNING XPRA PROCESSES
   ✅ Found 4 Xpra processes:
   1. bdx      1290492  0.0  0.1 247572 83192 ?        S    00:52   0:01 /usr/bin/Xvfb-for-Xpra-S1290455 +extension GLX +extension Composite -screen 0 1920x1080x24+32 -dpi 96 -nolisten tcp -noreset -displayfd 5
   2. bdx      1294507  0.0  0.1 247560 83488 ?        S    01:01   0:01 /usr/bin/Xvfb-for-Xpra-S1294475 +extension GLX +extension Composite -screen 0 1920x1080x24+32 -dpi 96 -nolisten tcp -noreset -displayfd 5
   3. bdx      1309284  0.0  0.1 247576 83732 ?        S    01:44   0:01 /usr/bin/Xvfb-for-Xpra-S1309252 +extension GLX +extension Composite -screen 0 1920x1080x24+32 -dpi 96 -nolisten tcp -noreset -displayfd 5
   4. bdx      1322530  0.0  0.1 237960 73540 ?        S    03:42   0:00 /usr/bin/Xvfb-for-Xpra-S1322498 +extension Composite -screen 0 1280x800x24+32 -noliste

In [8]:
def test_firefox_handler_logic():
    """Test the specific logic issues in firefox_handler.py that might prevent multi-sessions."""
    print("🔧 TESTING FIREFOX HANDLER LOGIC")
    print("=" * 40)
    
    # Check for the startup lock issue
    print("\n1. 🔒 CHECKING STARTUP LOCK INITIALIZATION")
    
    # Simulate the firefox handler class initialization logic
    class MockFirefoxHandler:
        _xpra_startup_lock = None
        _active_sessions = {}
        
        def __init__(self):
            # This is the problematic pattern from the original code
            if MockFirefoxHandler._xpra_startup_lock is None:
                import asyncio
                MockFirefoxHandler._xpra_startup_lock = asyncio.Lock()
            
            if not hasattr(MockFirefoxHandler, '_active_sessions'):
                MockFirefoxHandler._active_sessions = {}
                
        async def simulate_post_request(self, session_id):
            """Simulate what happens when multiple POST requests come in."""
            print(f"   📥 Session {session_id}: Starting POST request")
            
            try:
                # This is where the issue might be - using async with on a potentially None lock
                if MockFirefoxHandler._xpra_startup_lock is None:
                    print(f"   ❌ Session {session_id}: Lock is None!")
                    return False
                    
                async with MockFirefoxHandler._xpra_startup_lock:
                    print(f"   🔒 Session {session_id}: Acquired lock")
                    
                    # Simulate the session creation process
                    await asyncio.sleep(0.1)  # Simulate startup time
                    
                    port = 8080 + session_id
                    MockFirefoxHandler._active_sessions[port] = {
                        'process_id': 12300 + session_id,
                        'port': port
                    }
                    
                    print(f"   ✅ Session {session_id}: Created on port {port}")
                    print(f"   📊 Total active sessions: {len(MockFirefoxHandler._active_sessions)}")
                    return True
                    
            except Exception as e:
                print(f"   ❌ Session {session_id}: Error - {e}")
                return False
    
    # Test concurrent session creation
    async def test_concurrent_sessions():
        print("\n2. 🚀 TESTING CONCURRENT SESSION CREATION")
        
        # Create handler instances (simulating multiple requests)
        handlers = [MockFirefoxHandler() for _ in range(3)]
        
        # Try to create sessions concurrently
        tasks = []
        for i, handler in enumerate(handlers):
            task = handler.simulate_post_request(i + 1)
            tasks.append(task)
        
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        print(f"\n📊 CONCURRENT TEST RESULTS:")
        success_count = sum(1 for r in results if r is True)
        error_count = sum(1 for r in results if isinstance(r, Exception))
        
        print(f"   ✅ Successful sessions: {success_count}")
        print(f"   ❌ Failed sessions: {len(results) - success_count}")
        print(f"   🚨 Exceptions: {error_count}")
        
        if error_count > 0:
            print(f"\n🚨 EXCEPTIONS ENCOUNTERED:")
            for i, result in enumerate(results):
                if isinstance(result, Exception):
                    print(f"   Session {i+1}: {type(result).__name__}: {result}")
        
        return success_count == len(handlers)
    
    # Run the test using a thread to handle the event loop
    import asyncio
    import threading
    import queue
    
    result_queue = queue.Queue()
    
    def run_test():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            result = loop.run_until_complete(test_concurrent_sessions())
            result_queue.put(('success', result))
        except Exception as e:
            result_queue.put(('error', e))
        finally:
            loop.close()
    
    thread = threading.Thread(target=run_test)
    thread.start()
    thread.join()
    
    status, result = result_queue.get()
    if status == 'error':
        print(f"\n❌ TEST FAILED: {result}")
        print("💡 This suggests there are issues with the async lock implementation")
        return False
    
    concurrent_success = result
    
    print(f"\n🏁 FINAL ASSESSMENT:")
    if concurrent_success:
        print("   ✅ Multi-session logic appears to work correctly")
        print("   💡 Issue might be in Xpra configuration or port conflicts")
    else:
        print("   ❌ Multi-session logic has issues")
        print("   💡 Check the firefox_handler.py implementation")
    
    return concurrent_success

# Run the handler logic test
handler_test_result = test_firefox_handler_logic()

🔧 TESTING FIREFOX HANDLER LOGIC

1. 🔒 CHECKING STARTUP LOCK INITIALIZATION

2. 🚀 TESTING CONCURRENT SESSION CREATION
   📥 Session 1: Starting POST request
   🔒 Session 1: Acquired lock
   📥 Session 2: Starting POST request
   📥 Session 3: Starting POST request
   ✅ Session 1: Created on port 8081
   📊 Total active sessions: 1
   🔒 Session 2: Acquired lock
   ✅ Session 2: Created on port 8082
   📊 Total active sessions: 2
   🔒 Session 3: Acquired lock
   ✅ Session 3: Created on port 8083
   📊 Total active sessions: 3

📊 CONCURRENT TEST RESULTS:
   ✅ Successful sessions: 3
   ❌ Failed sessions: 0
   🚨 Exceptions: 0

🏁 FINAL ASSESSMENT:
   ✅ Multi-session logic appears to work correctly
   💡 Issue might be in Xpra configuration or port conflicts


In [9]:
def analyze_firefox_handler_issues():
    """Analyze the actual firefox_handler.py for multi-session blocking issues."""
    print("🔍 ANALYZING FIREFOX HANDLER IMPLEMENTATION")
    print("=" * 50)
    
    handler_path = Path.home() / 'allcode' / 'github' / 'vantagecompute' / 'jfl' / 'jupyterlab_firefox_launcher' / 'firefox_handler.py'
    
    if not handler_path.exists():
        print(f"❌ Firefox handler not found at: {handler_path}")
        return
    
    print(f"✅ Found firefox_handler.py at: {handler_path}")
    
    # Read and analyze the handler
    with open(handler_path, 'r') as f:
        content = f.read()
    
    # Check for specific issues
    issues = []
    
    # 1. Check for startup lock initialization
    print("\n1. 🔒 CHECKING STARTUP LOCK INITIALIZATION")
    if '_xpra_startup_lock = None' in content:
        print("   ✅ Found startup lock class variable")
        
        # Check if it's properly initialized
        if 'FirefoxLauncherHandler._xpra_startup_lock = asyncio.Lock()' in content:
            print("   ✅ Lock is properly initialized")
        else:
            print("   ⚠️ Lock initialization might be missing")
            issues.append("Startup lock may not be properly initialized")
    
    # 2. Check for the problematic async with pattern
    print("\n2. 🚨 CHECKING FOR ASYNC LOCK USAGE ISSUES")
    if 'async with FirefoxLauncherHandler._xpra_startup_lock:' in content:
        # This is the problematic pattern - let's see if there's a None check
        if 'if FirefoxLauncherHandler._xpra_startup_lock is None:' not in content:
            print("   ❌ FOUND ISSUE: async with used without None check!")
            issues.append("CRITICAL: async with used on potentially None lock")
        else:
            print("   ✅ Found None check before async with")
    
    # 3. Check for session cleanup interference
    print("\n3. 🧹 CHECKING FOR CLEANUP INTERFERENCE")
    cleanup_calls = content.count('_cleanup_inactive_sessions()')
    if cleanup_calls > 0:
        print(f"   ⚠️ Found {cleanup_calls} cleanup calls - this could block multi-sessions!")
        issues.append(f"Found {cleanup_calls} cleanup calls that could interfere")
    else:
        print("   ✅ No cleanup calls found in main handlers")
    
    # 4. Check for port conflicts
    print("\n4. 🌐 CHECKING PORT ALLOCATION LOGIC")
    if '_find_free_port()' in content:
        print("   ✅ Using dynamic port allocation")
    else:
        print("   ⚠️ Static port allocation detected")
        issues.append("Static port allocation could cause conflicts")
    
    # 5. Check for session tracking issues
    print("\n5. 📊 CHECKING SESSION TRACKING")
    if '_active_sessions' in content:
        print("   ✅ Found session tracking dictionary")
        
        # Check if it's a class variable
        if '_active_sessions: Dict[Any,Any] = {}' in content:
            print("   ✅ Session tracking is a class variable (shared across instances)")
        else:
            print("   ⚠️ Session tracking might be instance-specific")
            issues.append("Session tracking may not be shared across handler instances")
    
    # 6. Check for specific error patterns
    print("\n6. 🚨 CHECKING FOR KNOWN ERROR PATTERNS")
    
    # Check for the None lock error
    if 'Object of type "None" cannot be used with "async with"' in content:
        print("   ❌ FOUND: Known None lock error in comments/errors")
        issues.append("Code has known None lock async with issue")
    
    # Check if there are any synchronous blocking calls
    blocking_calls = ['subprocess.run(', 'time.sleep(', '.wait(']
    found_blocking = []
    for call in blocking_calls:
        if call in content and 'await' not in content[content.find(call):content.find(call)+50]:
            found_blocking.append(call.strip('('))
    
    if found_blocking:
        print(f"   ⚠️ Found potentially blocking calls: {found_blocking}")
        issues.append(f"Blocking calls found: {found_blocking}")
    
    # Summary
    print(f"\n🏁 ANALYSIS RESULTS:")
    if not issues:
        print("   ✅ No obvious implementation issues found")
        print("   💡 Issue might be in Xpra configuration or system-level conflicts")
    else:
        print(f"   ❌ Found {len(issues)} potential issues:")
        for i, issue in enumerate(issues, 1):
            print(f"   {i}. {issue}")
    
    # Specific recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    
    if any("CRITICAL" in issue for issue in issues):
        print("   🚨 CRITICAL FIXES NEEDED:")
        print("      - Fix the async with None lock issue")
        print("      - Add proper None checks before async with statements")
    
    if any("cleanup" in issue.lower() for issue in issues):
        print("   🧹 CLEANUP ISSUES:")
        print("      - Remove or disable cleanup calls from main handlers")
        print("      - Only cleanup when explicitly requested")
    
    print("   🔧 GENERAL DEBUGGING:")
    print("      - Enable detailed logging in firefox_handler.py")
    print("      - Test with a single session first")
    print("      - Check JupyterLab logs for specific error messages")
    
    return issues

# Run the analysis
handler_issues = analyze_firefox_handler_issues()

🔍 ANALYZING FIREFOX HANDLER IMPLEMENTATION
✅ Found firefox_handler.py at: /home/bdx/allcode/github/vantagecompute/jfl/jupyterlab_firefox_launcher/firefox_handler.py

1. 🔒 CHECKING STARTUP LOCK INITIALIZATION
   ✅ Found startup lock class variable
   ✅ Lock is properly initialized

2. 🚨 CHECKING FOR ASYNC LOCK USAGE ISSUES
   ✅ Found None check before async with

3. 🧹 CHECKING FOR CLEANUP INTERFERENCE
   ✅ No cleanup calls found in main handlers

4. 🌐 CHECKING PORT ALLOCATION LOGIC
   ✅ Using dynamic port allocation

5. 📊 CHECKING SESSION TRACKING
   ✅ Found session tracking dictionary
   ✅ Session tracking is a class variable (shared across instances)

6. 🚨 CHECKING FOR KNOWN ERROR PATTERNS
   ⚠️ Found potentially blocking calls: ['subprocess.run', 'time.sleep', '.wait']

🏁 ANALYSIS RESULTS:
   ❌ Found 1 potential issues:
   1. Blocking calls found: ['subprocess.run', 'time.sleep', '.wait']

💡 RECOMMENDATIONS:
   🔧 GENERAL DEBUGGING:
      - Enable detailed logging in firefox_handler.p

In [ ]:
def test_xpra_launching():
    """Test the actual Xpra launching process to identify blocking issues."""
    print("🚀 TESTING XPRA LAUNCHING PROCESS")
    print("=" * 40)
    
    # Test if we can find a free port
    print("\n1. 🌐 TESTING PORT ALLOCATION")
    try:
        import socket
        def find_free_port():
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('', 0))
                s.listen(1)
                port = s.getsockname()[1]
            return port
        
        ports = [find_free_port() for _ in range(3)]
        print(f"   ✅ Found free ports: {ports}")
        
        # Check if ports are actually free
        for port in ports:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                result = s.connect_ex(('localhost', port))
                if result == 0:
                    print(f"   ⚠️ Port {port} appears to be in use!")
                else:
                    print(f"   ✅ Port {port} is free")
                    
    except Exception as e:
        print(f"   ❌ Port allocation test failed: {e}")
    
    # Test Xpra command creation
    print("\n2. 🔧 TESTING XPRA COMMAND CREATION")
    try:
        # Simulate the command creation process
        from pathlib import Path
        
        test_port = 8555  # Use a specific test port
        session_dir = Path.home() / '.firefox-launcher' / 'sessions' / f'session-{test_port}'
        
        print(f"   📁 Test session directory: {session_dir}")
        
        # Check if we can create the session directory
        session_dir.mkdir(parents=True, exist_ok=True)
        if session_dir.exists():
            print(f"   ✅ Session directory created successfully")
        else:
            print(f"   ❌ Failed to create session directory")
            
        # Check for firefox-xstartup script
        from shutil import which
        firefox_wrapper = which('firefox-xstartup')
        if not firefox_wrapper:
            # Check development location
            firefox_wrapper = Path.home() / 'allcode' / 'github' / 'vantagecompute' / 'jfl' / 'scripts' / 'firefox-xstartup'
            if firefox_wrapper.exists():
                print(f"   ✅ Found firefox-xstartup at: {firefox_wrapper}")
            else:
                print(f"   ❌ firefox-xstartup not found")
                return False
        else:
            print(f"   ✅ Found firefox-xstartup in PATH: {firefox_wrapper}")
            
    except Exception as e:
        print(f"   ❌ Command creation test failed: {e}")
        return False
    
    # Test a simple Xpra command (without actually starting it)
    print("\n3. 🧪 TESTING XPRA COMMAND VALIDATION")
    try:
        # Check if xpra is available
        xpra = which('xpra')
        if not xpra:
            print("   ❌ Xpra not found in PATH")
            return False
        
        print(f"   ✅ Found Xpra: {xpra}")
        
        # Test xpra version (this should be quick)
        result = subprocess.run([xpra, '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"   ✅ Xpra version check successful: {result.stdout.strip()}")
        else:
            print(f"   ⚠️ Xpra version check failed: {result.stderr}")
            
        # Test firefox availability
        firefox = which('firefox')
        if firefox:
            print(f"   ✅ Found Firefox: {firefox}")
        else:
            print(f"   ❌ Firefox not found in PATH")
            
    except subprocess.TimeoutExpired:
        print("   ⚠️ Xpra version check timed out")
    except Exception as e:
        print(f"   ❌ Xpra validation failed: {e}")
        
    # Identify the blocking issue
    print("\n4. 🚨 IDENTIFYING BLOCKING ISSUES")
    
    blocking_issues = []
    
    # Check if there are long-running startup checks
    print("   📊 Analyzing potential blocking operations:")
    print("      - Port allocation: Fast ✅")
    print("      - Directory creation: Fast ✅") 
    print("      - Xpra version check: Fast ✅")
    print("      - Process startup monitoring: POTENTIALLY SLOW ⚠️")
    
    blocking_issues.append("Process startup monitoring with multiple sleep calls")
    blocking_issues.append("Synchronous subprocess.Popen with immediate polling")
    blocking_issues.append("Time.sleep() calls in startup monitoring")
    
    print(f"\n🏁 BLOCKING ANALYSIS RESULTS:")
    print(f"   ❌ Found {len(blocking_issues)} potential blocking issues:")
    for i, issue in enumerate(blocking_issues, 1):
        print(f"   {i}. {issue}")
        
    print(f"\n💡 SOLUTIONS TO ENABLE MULTI-SESSIONS:")
    print("   1. 🚀 Make process startup asynchronous")
    print("   2. ⏱️ Replace time.sleep() with asyncio.sleep()")
    print("   3. 🔄 Use non-blocking process monitoring")
    print("   4. 🎯 Return immediately after process creation")
    print("   5. 🧵 Use background tasks for startup verification")
    
    return True

# Run the Xpra launching test
xpra_test_result = test_xpra_launching()

🚀 TESTING XPRA LAUNCHING PROCESS

1. 🌐 TESTING PORT ALLOCATION
   ✅ Found free ports: [52897, 56229, 58207]
   ✅ Port 52897 is free
   ✅ Port 56229 is free
   ✅ Port 58207 is free

2. 🔧 TESTING XPRA COMMAND CREATION
   📁 Test session directory: /home/bdx/.firefox-launcher/sessions/session-8555
   ✅ Session directory created successfully
   ✅ Found firefox-xstartup in PATH: /home/bdx/allcode/github/vantagecompute/jfl/.venv/bin/firefox-xstartup

3. 🧪 TESTING XPRA COMMAND VALIDATION
   ✅ Found Xpra: /usr/bin/xpra
   ✅ Xpra version check successful: xpra v3.1.5
   ✅ Found Firefox: /usr/bin/firefox

4. 🚨 IDENTIFYING BLOCKING ISSUES
   📊 Analyzing potential blocking operations:
      - Port allocation: Fast ✅
      - Directory creation: Fast ✅
      - Xpra version check: Fast ✅
      - Process startup monitoring: POTENTIALLY SLOW ⚠️

🏁 BLOCKING ANALYSIS RESULTS:
   ❌ Found 3 potential blocking issues:
   1. Process startup monitoring with multiple sleep calls
   2. Synchronous subprocess.Pop

: 